In [1]:
%matplotlib inline

In [2]:
#from __future__ import division
import pandas as pd
import numpy as np
from ggplot import *

import os
import sys

In [3]:
# Sample titles with corresponding barcodes
# s9: WT
# s9+bcm: WT +BCM
# s17: triple sRNA mutant

samples = {
    's9': ['ATCACG', 'ACAGTG'],
    's9+bcm': ['CGATGT', 'GCCAAT'],
    's17': ['TTAGGC', 'GATCAG'],
    }

# Barcodes
barcodes = ['ATCACG', 'ACAGTG', 'CGATGT', 'GCCAAT', 'TTAGGC', 'GATCAG']

In [4]:
def extract_gff(line, ftype='gene'):
    rec = {}
    fields = line.strip().split('\t')
    if fields[2] != ftype:
        return None
    opts = {}
    for f in fields[8].split(';'):
        k,v = f.split('=')
        opts[k] = v
        
    rec.update({
            'gene': opts['gene'],
            'start': int(fields[3]),
            'end': int(fields[4]),
            'strand': fields[6]
        })
    return rec


def extract_bed(line):
    rec = {}
    fields = line.strip().split('\t')
    rec.update({
            'TU': ''.join([x for x in fields[3].split('-')[2:]]),
            'start': int(fields[1]),
            'end': int(fields[2]),
            'strand': fields[5]
        })
    return rec



def anno_df(annofile, extract=extract_gff):
    result = []
    with open(annofile) as fi:
        for line in fi:
            if not line.startswith('#'):
                rec = extract(line)
                if rec:
                    result.append(rec)
    return pd.DataFrame.from_records(result)

In [5]:
gff = anno_df('../ref/NC_000913.gff')
gff

,end,gene,start,strand
0,255,thrL,190,+
1,2799,thrA,337,+
2,3733,thrB,2801,+
3,5020,thrC,3734,+
4,5530,yaaX,5234,+
5,6459,yaaA,5683,-
6,7959,yaaJ,6529,-
7,9191,talB,8238,+
8,9893,mog,9306,+
9,10494,yaaH,9928,-


In [6]:
df = pd.read_table('../ref/UTR_5_3_sequence.new.bed', sep='\t',
                            header=None,
                            names=['TSS', 'TU_name', 'coord_3', 'coord_5', 'first_gene_3',
                                   'first_gene_5', 'first_gene_name', 'operon', 'promoter',
                                   'seq_5UTR', 'strand'])
df

,TSS,TU_name,coord_3,coord_5,first_gene_3,first_gene_5,first_gene_name,operon,promoter,seq_5UTR,strand
0,148,thrLABC,190,148,255,190,thrL,thrLABC,thrLp,ATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCA,forward
1,148,thrL,190,148,255,190,thrL,thrLABC,thrLp,ATAGCGCACAGACAGATAAAAATTACAGAGTACACAACATCCA,forward
2,5030,yaaX,5234,5030,5530,5234,yaaX,yaaX,yaaXp4,ATTATCTCAATCAGGCCGGGTTTGCTTTTATGCAGCCCGGCTTTTT...,forward
3,6587,yaaA,6587,6459,6459,5683,yaaA,yaaA,yaaAp3,ATCCGGATATCGGTCGCCAGCTTTCTCCGGACGCGTGGGATGATGT...,reverse
4,6615,yaaA,6615,6459,6459,5683,yaaA,yaaA,yaaAp5,GTGCGCCCGGTGTTTGATCCATTGCGTTATCCGGATATCGGTCGCC...,reverse
5,8017,yaaJ,8017,7959,7959,6529,yaaJ,yaaJ,yaaJp4,AACAATTGGTAACGTTTACACAGGAAAGTCATCGCGACCGGCAATA...,reverse
6,8191,talB,8238,8191,9191,8238,talB,talB,talBp,AGACCGGTTACATCCCCCTAACAAGCTGTTTAAAGAGAAATACTATCA,forward
7,10643,htgA,10830,10643,11315,10830,htgA,htgA,htgAp2,TCAGACCTGAGTGGCGCTAACCATCCGGCGCAGGCAGGCGATTTGC...,forward
8,10644,htgA,10830,10644,11315,10830,htgA,htgA,htgAp1,CAGACCTGAGTGGCGCTAACCATCCGGCGCAGGCAGGCGATTTGCA...,forward
9,11542,yaaW,11542,11356,11356,10643,yaaW,yaaW,yaaWp3,GCCTGAATATTCCTTCAGAAATAAAAGAAGGGCAAACCACTGACTG...,reverse


In [7]:
utrs = df[['TSS', 'coord_3', 'coord_5', 'first_gene_name', 'strand']].copy()
utrs.rename(columns={'coord_3': 'end', 'coord_5': 'start', 'first_gene_name': 'gene'}, inplace=True)
utrs.loc[utrs['strand'] == 'forward', 'strand'] = '+'
utrs.loc[utrs['strand'] == 'reverse', 'strand'] = '-'
utrs

,TSS,end,start,gene,strand
0,148,190,148,thrL,+
1,148,190,148,thrL,+
2,5030,5234,5030,yaaX,+
3,6587,6587,6459,yaaA,-
4,6615,6615,6459,yaaA,-
5,8017,8017,7959,yaaJ,-
6,8191,8238,8191,talB,+
7,10643,10830,10643,htgA,+
8,10644,10830,10644,htgA,+
9,11542,11542,11356,yaaW,-


In [8]:
# 5' UTR annotations
"""
res = []
with open('../../results/redux/utrs_corr.bed', 'r') as fi:
    for line in fi:
        fields = line.strip().split()
        res.append({
                'gene': fields[3],
                'start': int(fields[1]),
                'end': int(fields[2]),
            })
 
utrs = pd.DataFrame.from_records(res)
utrs['UTR_length'] = utrs.end - utrs.start
utrs
"""

"\nres = []\nwith open('../../results/redux/utrs_corr.bed', 'r') as fi:\n    for line in fi:\n        fields = line.strip().split()\n        res.append({\n                'gene': fields[3],\n                'start': int(fields[1]),\n                'end': int(fields[2]),\n            })\n \nutrs = pd.DataFrame.from_records(res)\nutrs['UTR_length'] = utrs.end - utrs.start\nutrs\n"

In [9]:
dfa = utrs.merge(gff, on='gene', how='left').dropna()
dfa['strand'] = dfa['strand_x']
dfa

,TSS,end_x,start_x,gene,strand_x,end_y,start_y,strand_y,strand
0,148,190,148,thrL,+,255.0,190.0,+,+
1,148,190,148,thrL,+,255.0,190.0,+,+
2,5030,5234,5030,yaaX,+,5530.0,5234.0,+,+
3,6587,6587,6459,yaaA,-,6459.0,5683.0,-,-
4,6615,6615,6459,yaaA,-,6459.0,5683.0,-,-
5,8017,8017,7959,yaaJ,-,7959.0,6529.0,-,-
6,8191,8238,8191,talB,+,9191.0,8238.0,+,+
9,11542,11542,11356,yaaW,-,11356.0,10643.0,-,-
10,11825,11825,11786,yaaI,-,11786.0,11382.0,-,-
11,11913,11913,11786,yaaI,-,11786.0,11382.0,-,-


In [10]:
import pysam


def get_ratio(df, bamfile, reference, win=50, offset=200):

    def _utr_orf_ratio(rec):
        
        def _counts(start, end):
            count = 1
            #start = int(rec[key5])
            #end = int(rec[key3])
            #if end < start:
            #    start, end = end, start
            for read in bam.fetch(reference, start, end):
                count += 1
            return float(count)
        
        if rec['strand'] == '-':
            end_utr = rec['end_x']
            start_utr = end_utr - win
            end_orf = end_utr - offset
            start_orf = end_orf - win
        else:
            start_utr = rec['start_x']
            end_utr = start_utr + win
            start_orf = rec['start_x'] + offset
            end_orf = start_orf + win
            
        return _counts(start_utr, end_utr) / _counts(start_orf, end_orf)

    bam = pysam.AlignmentFile(bamfile, 'rb')
    return df.apply(_utr_orf_ratio, axis=1)




def get_utr_ratio_df(data, barcodes, res_dir='../results', win=50, offset=200):
    '''
    Calculates 5'UTR coverage
    Iterates over files whose name ends with '_sorted.bam' in `res_dir` and contains barcodes
    specified in `barcodes`.
    
    Adds `utr_<barcode>` column to df DataFrame
    '''
    d, _, filenames = next(os.walk(res_dir))
    infiles = [f for f in filenames if f.endswith('_sorted.bam')]
    df = data.copy()
    for barcode in barcodes:
        bamfile = os.path.join(d, [f for f in infiles if barcode in f][0])
        df['ratio_{0}'.format(barcode)] = get_ratio(df,
                                            bamfile, 
                                            'gi|556503834|ref|NC_000913.3|',
                                            win, offset)
    output.put({'data': df, 'winsize': win, 'offset': offset})


In [11]:
import multiprocessing as mp


output = mp.Queue()

offsets = [150,200,300]
winsizes = [50,80,100,200]
output_tpl = '../results/dfa_mp.offset_{}.win_{}.csv'


def utr_ratio(data, barcodes, res_dir='../results', win=50, offset=200):
    '''
    Calculates 5'UTR coverage
    Iterates over files whose name ends with '_sorted.bam' in `res_dir` and contains barcodes
    specified in `barcodes`.
    
    Adds `utr_<barcode>` column to df DataFrame
    '''
    d, _, filenames = next(os.walk(res_dir))
    infiles = [f for f in filenames if f.endswith('_sorted.bam')]
    df = data.copy()
    for barcode in barcodes:
        bamfile = os.path.join(d, [f for f in infiles if barcode in f][0])
        df['ratio_{0}'.format(barcode)] = get_ratio(df,
                                            bamfile, 
                                            'gi|556503834|ref|NC_000913.3|',
                                            win, offset)
    df.to_csv(output_tpl.format(offset, win))
    output.put({'winsize': win, 'offset': offset, 'filename': output_tpl.format(offset, win)})

    
pool = []

for offset in offsets:
    for winsize in winsizes:
        d = dfa.copy()
        pool.append(mp.Process(target=utr_ratio,
                               args=(d, ['ATCACG', 'ACAGTG', 'CGATGT', 'GCCAAT']),
                                kwargs={'res_dir': '../results/',
                                        'offset': offset,
                                        'win': winsize}))
        
for p in pool:
    p.start()
    
for p in pool:
    p.join()
    
    
results = [output.get() for p in pool]

In [12]:
results

[{'filename': '../../results/redux/dfa_mp.offset_150.win_50.csv',
  'offset': 150,
  'winsize': 50},
 {'filename': '../../results/redux/dfa_mp.offset_200.win_50.csv',
  'offset': 200,
  'winsize': 50},
 {'filename': '../../results/redux/dfa_mp.offset_300.win_50.csv',
  'offset': 300,
  'winsize': 50},
 {'filename': '../../results/redux/dfa_mp.offset_150.win_80.csv',
  'offset': 150,
  'winsize': 80},
 {'filename': '../../results/redux/dfa_mp.offset_300.win_100.csv',
  'offset': 300,
  'winsize': 100},
 {'filename': '../../results/redux/dfa_mp.offset_300.win_80.csv',
  'offset': 300,
  'winsize': 80},
 {'filename': '../../results/redux/dfa_mp.offset_200.win_80.csv',
  'offset': 200,
  'winsize': 80},
 {'filename': '../../results/redux/dfa_mp.offset_150.win_100.csv',
  'offset': 150,
  'winsize': 100},
 {'filename': '../../results/redux/dfa_mp.offset_200.win_100.csv',
  'offset': 200,
  'winsize': 100},
 {'filename': '../../results/redux/dfa_mp.offset_300.win_200.csv',
  'offset': 300,
 

In [ ]:
output.get()